In [32]:
system_prompt ='''
请根据用户提供的自然语言描述生成一个MongoDB的查询对象，除非用户明确指出待查询的关键字为精确匹配，否则查询对象应支持模糊匹配。
用户输入的内容放置在<input></input>标签中，根据以下步骤生成查询对象：
1. 首先识别用户是否是询问了关于某个问题，如“查找项目名称包含‘***’的记录”、“查找可能设备名称‘****’的信息”等，并提取关键词。
2. 如果用户提出的不是相关查询信息，则询问用户是否需要进一步说明，如“请问您要查找哪个项目的设备信息？”、“请问您要查找哪个设备的采购信息？”等。
3. 如果识别到关键字，将关键字进行提取并按照<requirements></requirements>中的要求输出符合mongodb查询语法的查询对象，最后将完整的查询对象输出在<query></query>标签中。
<requirements>
目标集合的名称为`equipment_collection`包含以下字段：`project_name`（字符串，项目名称）、`contract_number`（字符串，合同编号）、`contract_type`（字符串，合同类型）、\
`subitem_name`（字符串，子项名称）、`device_name`（字符串，设备名称）、`specification_material`（字符串，规格材质）、\
`manufacturer`（字符串，制造商）。
用户的需求是：查找与输入关键词相关的所有记录，关键词可能出现在上述任意字段中。
如果是模糊匹配要求（大小写不敏感），覆盖所有可能的数据，查询结果包含所有字段。
注意不要虚构任何数据并按照用户的需求进行查询。
构建的查询语句应该以`db.equipment_collection.find(${query_json})`为模板。
只需要提供完整的MongoDB查询对象`query_json`，输出在<query></query>标签中，完整的<query></query>标签由<result></result>包裹，确保语法正确。
</requirements>
'''

In [33]:
from ollama import Client
client = Client(
  host='http://192.168.43.41:11434',
  headers={'x-some-header': 'some-value'}
)
response = client.chat(model='qwq:latest', 
                       options={
                           'temperature':0,
                           "num_ctx": 4096,
                       },
                       messages=[
  {'role':'system', 'content': system_prompt},
  {
    'role': 'user',
    'content': f"<input>包钢有哪些哪些设备？</input>"+"<think>\n",
  },
])
print(response.message.content)

input_text = response.message.content

<think>
好的，我需要处理用户的查询：“包钢有哪些哪些设备？”。首先，用户的问题是询问包钢相关的设备信息，所以我要确定他们想查找的是设备名称包含“包钢”的记录。根据要求，除非明确指出要精确匹配，否则应该使用模糊匹配。

接下来，检查用户输入是否属于查询类型。这里明显是在请求数据，所以需要提取关键词。“包钢”应该是关键字，可能出现在各个字段中，比如project_name、device_name等。

然后按照requirements中的集合结构，目标是equipment_collection，包含多个字符串字段。需要构建一个MongoDB的查询对象，覆盖所有可能的字段，并且使用正则表达式进行模糊匹配（大小写不敏感）。

要确保每个相关字段都有$regex条件，模式设置为“包钢”，并且用$i标志忽略大小写。这样就能在所有指定字段中搜索包含“包钢”的记录。同时，用户没有要求精确匹配，所以必须使用正则表达式而不是确切的字符串比较。

最后，将查询对象格式化成正确的JSON结构，并放入<query>标签里。需要确认每个字段都正确应用了正则表达式，并且整个查询符合MongoDB的语法。这样生成的结果就能准确回应用户的请求了。
</think>

<result>
<query>{
  "$or": [
    { "project_name": { "$regex": "包钢", "$options": "i" } },
    { "contract_number": { "$regex": "包钢", "$options": "i" } },
    { "contract_type": { "$regex": "包钢", "$options": "i" } },
    { "subitem_name": { "$regex": "包钢", "$options": "i" } },
    { "device_name": { "$regex": "包钢", "$options": "i" } },
    { "specification_material": { "$regex": "包钢", "$options": "i" } },
    { "manufacturer": { "$regex": "包钢", "$options": "i" } }
  ]

In [6]:
import re
import json

def extract_last_query_content(text):
    pattern = r'<query>(.*?)</query>'
    matches = list(re.finditer(pattern, text, re.DOTALL))
    if matches:
        return matches[-1].group(1)  # 返回最后一个匹配的捕获组
    return None

In [7]:
query_text = extract_last_query_content(input_text)

In [8]:
query_text

'{"contract_number": "BGYRHS-2023-14"}'

In [9]:
from pymongo import MongoClient


# 连接到 MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['equipment_db']  # 数据库名

# 将字符串转换为字典
query_dict = json.loads(query_text)

# 执行查询
results = db.equipment_collection.find(query_dict)

# 打印结果
print("找到的文档数量:", len(list(results)))


找到的文档数量: 75


In [36]:
r = '''
<think>
好的，我需要处理用户的查询：“包钢有哪些哪些设备？”。首先，用户的问题是询问包钢相关的设备信息，所以我要确定他们想查找的是设备名称包含“包钢”的记录。根据要求，除非明确指出要精确匹配，否则应该使用模糊匹配。

接下来，检查用户输入是否属于查询类型。这里明显是在请求数据，所以需要提取关键词。“包钢”应该是关键字，可能出现在各个字段中，比如project_name、device_name等。

然后按照requirements中的集合结构，目标是equipment_collection，包含多个字符串字段。需要构建一个MongoDB的查询对象，覆盖所有可能的字段，并且使用正则表达式进行模糊匹配（大小写不敏感）。

要确保每个相关字段都有$regex条件，模式设置为“包钢”，并且用$i标志忽略大小写。这样就能在所有指定字段中搜索包含“包钢”的记录。同时，用户没有要求精确匹配，所以必须使用正则表达式而不是确切的字符串比较。

最后，将查询对象格式化成正确的JSON结构，并放入<query>标签里。需要确认每个字段都正确应用了正则表达式，并且整个查询符合MongoDB的语法。这样生成的结果就能准确回应用户的请求了。
</think>

<result>
<query>{
  "$or": [
    { "project_name": { "$regex": "包钢", "$options": "i" } },
    { "contract_number": { "$regex": "包钢", "$options": "i" } },
    { "contract_type": { "$regex": "包钢", "$options": "i" } },
    { "subitem_name": { "$regex": "包钢", "$options": "i" } },
    { "device_name": { "$regex": "包钢", "$options": "i" } },
    { "specification_material": { "$regex": "包钢", "$options": "i" } },
    { "manufacturer": { "$regex": "包钢", "$options": "i" } }
  ]
}</query>
</result>
'''

In [34]:
def extract_last_result_query(text):
    """
    从文本中提取最后一个完整的 <result><query>...</query></result> 标签内容
    
    参数:
        text (str): 输入的文本内容
        
    返回:
        str: 最后一个完整的 <query> 标签内的内容，如果没有找到则返回 None
    """
    # 使用正则表达式匹配所有 <result><query>...</query></result> 块
    pattern = r'<result>\s*<query>(.*?)</query>\s*</result>'
    matches = re.findall(pattern, text, re.DOTALL)
    
    # 如果找到匹配项，返回最后一个
    if matches:
        return matches[-1].strip()
    else:
        return None

In [38]:
extract_last_result_query(r)

'{\n  "$or": [\n    { "project_name": { "$regex": "包钢", "$options": "i" } },\n    { "contract_number": { "$regex": "包钢", "$options": "i" } },\n    { "contract_type": { "$regex": "包钢", "$options": "i" } },\n    { "subitem_name": { "$regex": "包钢", "$options": "i" } },\n    { "device_name": { "$regex": "包钢", "$options": "i" } },\n    { "specification_material": { "$regex": "包钢", "$options": "i" } },\n    { "manufacturer": { "$regex": "包钢", "$options": "i" } }\n  ]\n}'